# 💼 Casos de Uso Reales con Redis

*Autor: @mCárdenas 2025*

2 casos de uso prácticos por cada tipo de dato.

In [ ]:
import redis
import json
import time
import random
from datetime import datetime, timedelta

r = redis.Redis(host='localhost', port=6379, decode_responses=True)
print(f"✅ Conectado: {r.ping()}")

## 📝 STRINGS

### Caso 1: Contador de Visitas Web

In [ ]:
class ContadorVisitas:
    def __init__(self, redis_client):
        self.r = redis_client
    
    def registrar_visita(self, pagina: str):
        """Incrementa el contador de visitas para una página."""
        clave = f"visitas:{pagina}"
        return self.r.incr(clave)
    
    def obtener_visitas(self, pagina: str) -> int:
        """Obtiene el número de visitas de una página."""
        return int(self.r.get(f"visitas:{pagina}") or 0)
    
    def top_paginas(self, n: int = 5):
        """Obtiene las páginas más visitadas."""
        claves = self.r.keys("visitas:*")
        visitas = [(k.replace("visitas:", ""), int(self.r.get(k))) for k in claves]
        return sorted(visitas, key=lambda x: x[1], reverse=True)[:n]

# Demo
contador = ContadorVisitas(r)
paginas = ['/home', '/productos', '/contacto', '/blog', '/home', '/home', '/productos']

for pagina in paginas:
    visitas = contador.registrar_visita(pagina)
    print(f"📊 {pagina}: {visitas} visitas")

print("🏆 Top páginas:")
for pagina, visitas in contador.top_paginas():
    print(f"  {pagina}: {visitas}")

### Caso 2: Tokens de Sesión con TTL

In [ ]:
import secrets

class GestorSesiones:
    def __init__(self, redis_client, ttl_segundos=3600):
        self.r = redis_client
        self.ttl = ttl_segundos
    
    def crear_sesion(self, user_id: str, datos: dict) -> str:
        """Crea una nueva sesión y devuelve el token."""
        token = secrets.token_urlsafe(32)
        datos['user_id'] = user_id
        datos['creada'] = datetime.now().isoformat()
        self.r.setex(f"sesion:{token}", self.ttl, json.dumps(datos))
        return token
    
    def validar_sesion(self, token: str) -> dict:
        """Valida un token y devuelve los datos de sesión."""
        datos = self.r.get(f"sesion:{token}")
        if datos:
            self.r.expire(f"sesion:{token}", self.ttl)  # Renovar TTL
            return json.loads(datos)
        return None
    
    def cerrar_sesion(self, token: str):
        """Elimina una sesión."""
        return self.r.delete(f"sesion:{token}")

# Demo
gestor = GestorSesiones(r, ttl_segundos=60)

token = gestor.crear_sesion("user123", {"nombre": "María", "rol": "admin"})
print(f"🔑 Token creado: {token[:20]}...")

sesion = gestor.validar_sesion(token)
print(f"✅ Sesión válida: {sesion}")

print(f"⏱️ TTL restante: {r.ttl(f'sesion:{token}')} segundos")

## 📋 LISTS

### Caso 1: Cola de Tareas (Job Queue)

In [ ]:
class ColaTareas:
    def __init__(self, redis_client, nombre_cola="tareas"):
        self.r = redis_client
        self.cola = f"cola:{nombre_cola}"
    
    def encolar(self, tarea: dict):
        """Añade una tarea a la cola."""
        tarea['timestamp'] = datetime.now().isoformat()
        self.r.rpush(self.cola, json.dumps(tarea))
        return self.r.llen(self.cola)
    
    def procesar(self):
        """Obtiene y procesa la siguiente tarea (FIFO)."""
        tarea_json = self.r.lpop(self.cola)
        if tarea_json:
            return json.loads(tarea_json)
        return None
    
    def pendientes(self) -> int:
        return self.r.llen(self.cola)

# Demo
cola = ColaTareas(r, "emails")

# Productor: Encolar tareas
tareas = [
    {"tipo": "email", "destinatario": "user1@email.com", "asunto": "Bienvenido"},
    {"tipo": "email", "destinatario": "user2@email.com", "asunto": "Factura"},
    {"tipo": "email", "destinatario": "user3@email.com", "asunto": "Recordatorio"},
]

for tarea in tareas:
    pos = cola.encolar(tarea)
    print(f"📥 Encolada: {tarea['asunto']} (posición {pos})")

# Consumidor: Procesar tareas
print(f"📬 Procesando {cola.pendientes()} tareas:")
while cola.pendientes() > 0:
    tarea = cola.procesar()
    print(f"  ✅ Enviando a {tarea['destinatario']}: {tarea['asunto']}")

### Caso 2: Historial de Actividades

In [ ]:
class HistorialActividades:
    def __init__(self, redis_client, max_items=100):
        self.r = redis_client
        self.max_items = max_items
    
    def registrar(self, user_id: str, actividad: str):
        """Registra una actividad en el historial del usuario."""
        clave = f"historial:{user_id}"
        entrada = json.dumps({"actividad": actividad, "timestamp": datetime.now().isoformat()})
        self.r.lpush(clave, entrada)
        self.r.ltrim(clave, 0, self.max_items - 1)  # Mantener solo los últimos N
    
    def obtener(self, user_id: str, n: int = 10):
        """Obtiene las últimas N actividades."""
        clave = f"historial:{user_id}"
        items = self.r.lrange(clave, 0, n - 1)
        return [json.loads(item) for item in items]

# Demo
historial = HistorialActividades(r)

actividades = ["Login", "Ver productos", "Añadir al carrito", "Checkout", "Pago completado"]
for act in actividades:
    historial.registrar("user123", act)
    time.sleep(0.1)

print("📜 Últimas actividades de user123:")
for item in historial.obtener("user123", 5):
    print(f"  • {item['actividad']} ({item['timestamp']})")

## 🔵 SETS

### Caso 1: Sistema de Etiquetas de Productos

In [ ]:
class SistemaEtiquetas:
    def __init__(self, redis_client):
        self.r = redis_client
    
    def etiquetar(self, producto_id: str, *etiquetas):
        self.r.sadd(f"tags:{producto_id}", *etiquetas)
        for etiqueta in etiquetas:
            self.r.sadd(f"productos_con_tag:{etiqueta}", producto_id)
    
    def etiquetas_de(self, producto_id: str):
        return self.r.smembers(f"tags:{producto_id}")
    
    def productos_por_tag(self, etiqueta: str):
        return self.r.smembers(f"productos_con_tag:{etiqueta}")
    
    def productos_con_todos_los_tags(self, *etiquetas):
        claves = [f"productos_con_tag:{e}" for e in etiquetas]
        return self.r.sinter(*claves)

# Demo
tags = SistemaEtiquetas(r)
tags.etiquetar("prod1", "electrónica", "oferta", "nuevo")
tags.etiquetar("prod2", "electrónica", "premium")
tags.etiquetar("prod3", "hogar", "oferta")

print(f"🏷️ Tags de prod1: {tags.etiquetas_de('prod1')}")
print(f"📦 Productos con 'oferta': {tags.productos_por_tag('oferta')}")
print(f"📦 Electrónica + oferta: {tags.productos_con_todos_los_tags('electrónica', 'oferta')}")

### Caso 2: Sistema de Seguidores

In [ ]:
class RedSocial:
    def __init__(self, redis_client):
        self.r = redis_client
    
    def seguir(self, usuario: str, a_seguir: str):
        self.r.sadd(f"siguiendo:{usuario}", a_seguir)
        self.r.sadd(f"seguidores:{a_seguir}", usuario)
    
    def siguiendo(self, usuario: str):
        return self.r.smembers(f"siguiendo:{usuario}")
    
    def seguidores(self, usuario: str):
        return self.r.smembers(f"seguidores:{usuario}")
    
    def amigos_en_comun(self, user1: str, user2: str):
        return self.r.sinter(f"siguiendo:{user1}", f"siguiendo:{user2}")

# Demo
red = RedSocial(r)
red.seguir("ana", "carlos")
red.seguir("ana", "maria")
red.seguir("pedro", "carlos")
red.seguir("pedro", "maria")
red.seguir("pedro", "luis")

print(f"👤 Ana sigue a: {red.siguiendo('ana')}")
print(f"👥 Seguidores de Carlos: {red.seguidores('carlos')}")
print(f"🤝 Amigos en común (Ana y Pedro): {red.amigos_en_comun('ana', 'pedro')}")

## 🏆 SORTED SETS

### Caso 1: Leaderboard de Videojuego

In [ ]:
class Leaderboard:
    def __init__(self, redis_client, nombre="global"):
        self.r = redis_client
        self.key = f"leaderboard:{nombre}"
    
    def actualizar_puntuacion(self, jugador: str, puntos: int):
        self.r.zincrby(self.key, puntos, jugador)
    
    def top(self, n: int = 10):
        return self.r.zrevrange(self.key, 0, n-1, withscores=True)
    
    def posicion(self, jugador: str):
        rank = self.r.zrevrank(self.key, jugador)
        return rank + 1 if rank is not None else None
    
    def puntuacion(self, jugador: str):
        return self.r.zscore(self.key, jugador)

# Demo
lb = Leaderboard(r, "mundial")
jugadores = [("player1", 1500), ("player2", 2200), ("player3", 1800), ("player4", 3000), ("player5", 2500)]

for jugador, puntos in jugadores:
    lb.actualizar_puntuacion(jugador, puntos)

print("🏆 LEADERBOARD MUNDIAL:")
for i, (jugador, puntos) in enumerate(lb.top(5), 1):
    medalla = ['🥇', '🥈', '🥉'][i-1] if i <= 3 else f"#{i}"
    print(f"  {medalla} {jugador}: {int(puntos)} pts")

# Simular partida
lb.actualizar_puntuacion("player1", 2000)  # +2000 puntos
print(f"🎮 Player1 tras nueva partida: posición #{lb.posicion('player1')}, {int(lb.puntuacion('player1'))} pts")

### Caso 2: Feed de Noticias Ordenado

In [ ]:
class FeedNoticias:
    def __init__(self, redis_client):
        self.r = redis_client
    
    def publicar(self, usuario: str, post_id: str, contenido: str):
        timestamp = time.time()
        self.r.hset(f"post:{post_id}", mapping={"autor": usuario, "contenido": contenido})
        # Añadir al feed de los seguidores
        seguidores = self.r.smembers(f"seguidores:{usuario}")
        for seguidor in seguidores:
            self.r.zadd(f"feed:{seguidor}", {post_id: timestamp})
        # También al feed del autor
        self.r.zadd(f"feed:{usuario}", {post_id: timestamp})
    
    def obtener_feed(self, usuario: str, n: int = 10):
        post_ids = self.r.zrevrange(f"feed:{usuario}", 0, n-1)
        posts = []
        for post_id in post_ids:
            post = self.r.hgetall(f"post:{post_id}")
            post['id'] = post_id
            posts.append(post)
        return posts

# Demo
feed = FeedNoticias(r)
red.seguir("usuario1", "blogger1")  # Reutilizamos RedSocial

feed.publicar("blogger1", "post1", "¡Mi primer post!")
time.sleep(0.1)
feed.publicar("blogger1", "post2", "Aprendiendo Redis")

print("📰 Feed de usuario1:")
for post in feed.obtener_feed("usuario1"):
    print(f"  @{post['autor']}: {post['contenido']}")

## 🗂️ HASHES

### Caso 1: Perfil de Usuario

In [ ]:
class GestorUsuarios:
    def __init__(self, redis_client):
        self.r = redis_client
    
    def crear(self, user_id: str, datos: dict):
        datos['creado'] = datetime.now().isoformat()
        datos['visitas'] = 0
        self.r.hset(f"user:{user_id}", mapping=datos)
    
    def obtener(self, user_id: str):
        return self.r.hgetall(f"user:{user_id}")
    
    def actualizar(self, user_id: str, campo: str, valor):
        self.r.hset(f"user:{user_id}", campo, valor)
    
    def incrementar_visitas(self, user_id: str):
        return self.r.hincrby(f"user:{user_id}", "visitas", 1)

# Demo
usuarios = GestorUsuarios(r)
usuarios.crear("u001", {"nombre": "Ana García", "email": "ana@email.com", "plan": "premium"})

for _ in range(5):
    usuarios.incrementar_visitas("u001")

print("👤 Perfil de u001:")
for campo, valor in usuarios.obtener("u001").items():
    print(f"  {campo}: {valor}")

### Caso 2: Carrito de Compras

In [ ]:
class Carrito:
    def __init__(self, redis_client, user_id: str):
        self.r = redis_client
        self.key = f"carrito:{user_id}"
    
    def agregar(self, producto_id: str, cantidad: int = 1):
        return self.r.hincrby(self.key, producto_id, cantidad)
    
    def quitar(self, producto_id: str, cantidad: int = 1):
        nueva_cantidad = self.r.hincrby(self.key, producto_id, -cantidad)
        if nueva_cantidad <= 0:
            self.r.hdel(self.key, producto_id)
        return max(0, nueva_cantidad)
    
    def contenido(self):
        return {k: int(v) for k, v in self.r.hgetall(self.key).items()}
    
    def vaciar(self):
        self.r.delete(self.key)

# Demo
carrito = Carrito(r, "user123")
carrito.vaciar()

carrito.agregar("LAPTOP-001", 1)
carrito.agregar("MOUSE-002", 2)
carrito.agregar("TECLADO-003", 1)
carrito.quitar("MOUSE-002", 1)

print("🛒 Carrito de compras:")
for producto, cantidad in carrito.contenido().items():
    print(f"  {producto}: {cantidad} unidades")

## 🌊 STREAMS

### Caso 1: Log de Eventos en Tiempo Real

In [ ]:
class LogEventos:
    def __init__(self, redis_client, nombre="app"):
        self.r = redis_client
        self.stream = f"logs:{nombre}"
    
    def log(self, nivel: str, mensaje: str, **extra):
        evento = {"nivel": nivel, "mensaje": mensaje, "timestamp": datetime.now().isoformat()}
        evento.update(extra)
        return self.r.xadd(self.stream, evento)
    
    def ultimos(self, n: int = 10):
        return self.r.xrevrange(self.stream, count=n)

# Demo
logs = LogEventos(r, "mi_app")
r.delete("logs:mi_app")  # Limpiar

logs.log("INFO", "Aplicación iniciada")
logs.log("INFO", "Usuario conectado", user_id="u123")
logs.log("WARNING", "Intento de acceso fallido", ip="192.168.1.100")
logs.log("ERROR", "Error de base de datos", error="timeout")

print("📜 Últimos eventos:")
for id_evento, campos in logs.ultimos(5):
    print(f"  [{campos['nivel']}] {campos['mensaje']}")

### Caso 2: Chat Simple

In [ ]:
class SalaChat:
    def __init__(self, redis_client, sala: str):
        self.r = redis_client
        self.stream = f"chat:{sala}"
    
    def enviar(self, usuario: str, mensaje: str):
        return self.r.xadd(self.stream, {"usuario": usuario, "mensaje": mensaje})
    
    def historial(self, n: int = 50):
        return self.r.xrange(self.stream, count=n)

# Demo
chat = SalaChat(r, "general")
r.delete("chat:general")

chat.enviar("Ana", "Hola a todos!")
chat.enviar("Pedro", "Hola Ana!")
chat.enviar("Ana", "¿Qué tal el proyecto?")
chat.enviar("Pedro", "Bien, terminando con Redis")

print("💬 Chat #general:")
for _, msg in chat.historial():
    print(f"  <{msg['usuario']}> {msg['mensaje']}")

## 🔢 BITMAPS

### Caso 1: Registro de Asistencia Diaria

In [ ]:
class RegistroAsistencia:
    def __init__(self, redis_client):
        self.r = redis_client
    
    def _clave(self, fecha):
        return f"asistencia:{fecha}"
    
    def registrar(self, user_id: int, fecha: str = None):
        fecha = fecha or datetime.now().strftime("%Y-%m-%d")
        self.r.setbit(self._clave(fecha), user_id, 1)
    
    def asistio(self, user_id: int, fecha: str) -> bool:
        return bool(self.r.getbit(self._clave(fecha), user_id))
    
    def total_asistentes(self, fecha: str) -> int:
        return self.r.bitcount(self._clave(fecha))

# Demo
asistencia = RegistroAsistencia(r)
fecha_hoy = datetime.now().strftime("%Y-%m-%d")

for user_id in [1, 5, 10, 15, 20, 100, 500]:
    asistencia.registrar(user_id, fecha_hoy)

print(f"📅 Asistencia {fecha_hoy}:")
print(f"  Total: {asistencia.total_asistentes(fecha_hoy)} personas")
print(f"  ¿Usuario 10 asistió? {asistencia.asistio(10, fecha_hoy)}")
print(f"  ¿Usuario 11 asistió? {asistencia.asistio(11, fecha_hoy)}")

### Caso 2: Usuarios Activos por Día

In [ ]:
class UsuariosActivos:
    def __init__(self, redis_client):
        self.r = redis_client
    
    def marcar_activo(self, user_id: int, fecha: str = None):
        fecha = fecha or datetime.now().strftime("%Y-%m-%d")
        self.r.setbit(f"activos:{fecha}", user_id, 1)
    
    def activos_en_rango(self, fechas: list):
        """Usuarios activos en TODOS los días del rango."""
        claves = [f"activos:{f}" for f in fechas]
        self.r.bitop("AND", "activos:rango", *claves)
        return self.r.bitcount("activos:rango")

# Demo
activos = UsuariosActivos(r)

# Simular actividad
for uid in [1, 2, 3, 5, 10]:
    activos.marcar_activo(uid, "2024-01-15")
for uid in [1, 2, 4, 5]:
    activos.marcar_activo(uid, "2024-01-16")

print(f"👥 Activos ambos días: {activos.activos_en_rango(['2024-01-15', '2024-01-16'])}")

## 🎲 HYPERLOGLOG

### Caso 1: Visitantes Únicos

In [ ]:
class ContadorUnicos:
    def __init__(self, redis_client):
        self.r = redis_client
    
    def registrar(self, metrica: str, *valores):
        self.r.pfadd(f"hll:{metrica}", *valores)
    
    def contar(self, metrica: str) -> int:
        return self.r.pfcount(f"hll:{metrica}")
    
    def unir(self, destino: str, *metricas):
        claves = [f"hll:{m}" for m in metricas]
        self.r.pfmerge(f"hll:{destino}", *claves)

# Demo
contador = ContadorUnicos(r)

# Simular visitas
for i in range(1000):
    user = f"user_{random.randint(1, 500)}"  # ~500 únicos
    contador.registrar("visitas:hoy", user)

print(f"👥 Visitantes únicos (aproximado): {contador.contar('visitas:hoy')}")
print(f"💡 Memoria usada: ~12KB (constante, sin importar cantidad)")

### Caso 2: Métricas de Alcance

In [ ]:
class MetricasAlcance:
    def __init__(self, redis_client):
        self.r = redis_client
    
    def registrar_impresion(self, post_id: str, user_id: str):
        self.r.pfadd(f"alcance:{post_id}", user_id)
    
    def alcance(self, post_id: str) -> int:
        return self.r.pfcount(f"alcance:{post_id}")

# Demo
metricas = MetricasAlcance(r)

# Simular impresiones de un post viral
for _ in range(10000):
    user = f"user_{random.randint(1, 3000)}"
    metricas.registrar_impresion("post_viral", user)

print(f"📊 Alcance del post: ~{metricas.alcance('post_viral')} usuarios únicos")

## 📍 GEOSPATIAL

### Caso 1: Búsqueda de Restaurantes Cercanos

In [ ]:
class BuscadorRestaurantes:
    def __init__(self, redis_client):
        self.r = redis_client
        self.key = "geo:restaurantes"
    
    def agregar(self, nombre: str, lng: float, lat: float, datos: dict):
        self.r.geoadd(self.key, (lng, lat, nombre))
        self.r.hset(f"restaurante:{nombre}", mapping=datos)
    
    def buscar_cercanos(self, lng: float, lat: float, radio_km: float):
        resultados = self.r.geosearch(self.key, longitude=lng, latitude=lat, 
                                       radius=radio_km, unit='km', withdist=True)
        restaurantes = []
        for nombre, distancia in resultados:
            datos = self.r.hgetall(f"restaurante:{nombre}")
            datos['nombre'] = nombre
            datos['distancia_km'] = round(distancia, 2)
            restaurantes.append(datos)
        return restaurantes

# Demo
buscador = BuscadorRestaurantes(r)
buscador.agregar("Pizzería Italia", -3.7037, 40.4168, {"tipo": "italiana", "precio": "€€"})
buscador.agregar("Sushi Tokyo", -3.7000, 40.4200, {"tipo": "japonesa", "precio": "€€€"})
buscador.agregar("Bar Tapas", -3.7100, 40.4150, {"tipo": "española", "precio": "€"})

print("🍽️ Restaurantes cercanos a Puerta del Sol:")
for rest in buscador.buscar_cercanos(-3.7037, 40.4168, 2):
    print(f"  📍 {rest['nombre']} ({rest['tipo']}) - {rest['distancia_km']} km")

### Caso 2: Tracking de Vehículos

In [ ]:
class FlotaVehiculos:
    def __init__(self, redis_client):
        self.r = redis_client
        self.key = "geo:flota"
    
    def actualizar_posicion(self, vehiculo_id: str, lng: float, lat: float):
        self.r.geoadd(self.key, (lng, lat, vehiculo_id))
    
    def vehiculos_cercanos(self, lng: float, lat: float, radio_km: float):
        return self.r.geosearch(self.key, longitude=lng, latitude=lat,
                                radius=radio_km, unit='km', withdist=True, sort='ASC')

# Demo
flota = FlotaVehiculos(r)

# Actualizar posiciones
flota.actualizar_posicion("taxi-001", -3.7050, 40.4170)
flota.actualizar_posicion("taxi-002", -3.7020, 40.4165)
flota.actualizar_posicion("taxi-003", -3.7200, 40.4300)

print("🚕 Taxis cercanos al cliente:")
for vehiculo, dist in flota.vehiculos_cercanos(-3.7037, 40.4168, 1):
    print(f"  {vehiculo}: {dist:.2f} km")

## 📄 JSON

### Caso 1: Catálogo de Productos E-commerce

In [ ]:
try:
    class CatalogoProductos:
        def __init__(self, redis_client):
            self.r = redis_client
        
        def crear(self, producto_id: str, producto: dict):
            self.r.json().set(f"producto:json:{producto_id}", '$', producto)
        
        def obtener(self, producto_id: str):
            return self.r.json().get(f"producto:json:{producto_id}")
        
        def actualizar_stock(self, producto_id: str, delta: int):
            return self.r.json().numincrby(f"producto:json:{producto_id}", '$.stock', delta)
    
    catalogo = CatalogoProductos(r)
    catalogo.crear("P001", {
        "nombre": "MacBook Pro",
        "precio": 2499.99,
        "stock": 50,
        "specs": {"cpu": "M3 Pro", "ram": "18GB"},
        "colores": ["plata", "negro"]
    })
    
    print("📦 Producto:")
    print(json.dumps(catalogo.obtener("P001"), indent=2))
    
    catalogo.actualizar_stock("P001", -5)
    print(f"\n📦 Stock actualizado: {catalogo.obtener('P001')['stock']}")
except:
    print("⚠️ RedisJSON requiere RedisStack")

### Caso 2: Configuración de Aplicación

In [ ]:
try:
    class ConfiguracionApp:
        def __init__(self, redis_client, app_id: str):
            self.r = redis_client
            self.key = f"config:{app_id}"
        
        def inicializar(self, config: dict):
            self.r.json().set(self.key, '$', config)
        
        def obtener(self, path: str = '$'):
            return self.r.json().get(self.key, path)
        
        def actualizar(self, path: str, valor):
            self.r.json().set(self.key, path, valor)
    
    config = ConfiguracionApp(r, "mi_app")
    config.inicializar({
        "debug": False,
        "max_usuarios": 1000,
        "features": {"dark_mode": True, "beta": False}
    })
    
    print(f"⚙️ Debug: {config.obtener('$.debug')}")
    
    config.actualizar('$.features.beta', True)
    print(f"⚙️ Beta activado: {config.obtener('$.features.beta')}")
except:
    print("⚠️ RedisJSON requiere RedisStack")

## 🔍 SEARCH

### Caso 1: Buscador de Productos

In [ ]:
try:
    from redis.commands.search.field import TextField, NumericField, TagField
    from redis.commands.search.indexDefinition import IndexDefinition, IndexType
    
    # Limpiar y crear índice
    try:
        r.ft('idx:tienda').dropindex(delete_documents=True)
    except:
        pass
    
    r.ft('idx:tienda').create_index([
        TextField('nombre', sortable=True),
        TextField('descripcion'),
        NumericField('precio', sortable=True),
        TagField('categoria')
    ], definition=IndexDefinition(prefix=['item:'], index_type=IndexType.HASH))
    
    # Añadir productos
    productos = [
        {"nombre": "iPhone 15 Pro", "descripcion": "Smartphone Apple última generación", "precio": 1199, "categoria": "telefonia"},
        {"nombre": "Samsung Galaxy S24", "descripcion": "Smartphone Android premium", "precio": 999, "categoria": "telefonia"},
        {"nombre": "MacBook Air M3", "descripcion": "Portátil Apple ultraligero", "precio": 1299, "categoria": "portatiles"},
    ]
    
    for i, p in enumerate(productos, 1):
        r.hset(f"item:{i}", mapping=p)
    
    # Buscar
    print("🔍 Búsqueda: 'smartphone'")
    for doc in r.ft('idx:tienda').search('smartphone').docs:
        print(f"  • {doc.nombre} - ${doc.precio}")
    
    print("🔍 Búsqueda: precio < 1200")
    for doc in r.ft('idx:tienda').search('@precio:[0 1200]').docs:
        print(f"  • {doc.nombre} - ${doc.precio}")
except Exception as e:
    print(f"⚠️ RediSearch requiere RedisStack: {e}")

### Caso 2: Autocompletado

In [ ]:
try:
    # Añadir sugerencias
    sugerencias = ["iPhone", "iPhone 15", "iPhone 15 Pro", "iPad", "iPad Pro", "MacBook", "MacBook Pro", "MacBook Air"]
    
    for sug in sugerencias:
        r.ft('idx:tienda').sugadd('sugerencias', sug, 1.0)
    
    # Buscar sugerencias
    print("🔤 Autocompletado para 'mac':")
    for sug in r.ft('idx:tienda').sugget('sugerencias', 'mac', fuzzy=True):
        print(f"  → {sug.string}")
    
    print("\n🔤 Autocompletado para 'ip':")
    for sug in r.ft('idx:tienda').sugget('sugerencias', 'ip', fuzzy=True):
        print(f"  → {sug.string}")
except Exception as e:
    print(f"⚠️ RediSearch requiere RedisStack: {e}")

## ✅ Resumen

Has visto 2 casos de uso reales para cada tipo de dato de Redis. Estos patrones son utilizados en producción por empresas como Twitter, Instagram, Airbnb y muchas más.